##### 1단계: LangChain 설치 및 설정

gcloud cli 인증


In [1]:
#gcloud auth application-default login

In [2]:
from langchain_google_vertexai import ChatVertexAI
print("OK")

OK


##### 2단계: Groq Qwen2.5 32B 설치 및 설정 지침  -> 대신 GCP gemini로 변경

In [3]:
import os
from langchain_google_vertexai import ChatVertexAI


# Vertex AI Gemini 기반 LLM
llm = ChatVertexAI(
    model="gemini-2.5-flash",   # 또는 "gemini-2.5-pro"
    temperature=0.0,
    
)


##### 3단계: NVIDIA bge-m3 설치 및 설정

In [4]:
#pip install -qU langchain-nvidia-ai-endpoints


In [5]:

from langchain_google_vertexai import VertexAIEmbeddings

embeddings = VertexAIEmbeddings(model_name="text-embedding-004")

##### 4단계: Milvus 설치 및 설정

In [7]:
from langchain_milvus import Milvus
vector_store = Milvus(
    embedding_function=embeddings,
    collection_name="milvus_overview_docs",
    connection_args={"uri": "http://localhost:19530"},
)
# vector_store = Milvus(embedding_function=embeddings,
#     #                   collection_name="milvus_overview_docs",
#     # connection_args={"uri": "./milvus.db"},  # 현재 작업 디렉토리에 milvus.db라는 파일 하나를 만들고, 여기에 벡터들을 저장
                      
#                       )


MilvusException: <MilvusException: (code=2, message=Fail connecting to server on localhost:19530, illegal connection params or server unavailable)>

##### 5단계: RAG 챗봇 구축


이제 모든 구성 요소를 설정했으니 간단한 챗봇을 만들어 보겠습니다. Milvus 소개 문서를 비공개 지식 기반으로 활용하겠습니다. 이 문서를 사용자 정의 데이터 세트로 대체하여 RAG 챗봇을 맞춤 설정할 수 있습니다.

In [ ]:
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict

# Load and chunk contents of the blog
loader = WebBaseLoader(
    web_paths=("https://milvus.io/docs/overview.md",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("doc-style doc-post-content")
        )
    ),
)

docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
all_splits = text_splitter.split_documents(docs)

# Index chunks
_ = vector_store.add_documents(documents=all_splits)

# Define prompt for question-answering
prompt = hub.pull("rlm/rag-prompt")


# Define state for application
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str


# Define application steps
def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"])
    return {"context": retrieved_docs}


def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response.content}


# Compile application and test
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()


ModuleNotFoundError: No module named 'langchain'

##### 챗봇 테스트
네! 챗봇을 직접 만드셨네요. 챗봇에게 질문을 던져 볼까요?

In [ ]:
response = graph.invoke({"question": "What data types does Milvus support?"})
print(response["answer"])


gkdl
